# Dragon Framework Performance Comparison

This notebook compares the CPU-based and GPU-accelerated computation of voxelwise degree centrality.

In [ ]:
import time
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Set seed for reproducibility
np.random.seed(42)

# Parameters
n_voxels = 5000
n_timepoints = 200

# Generate synthetic data
data_cpu = np.random.randn(n_voxels, n_timepoints)
data_gpu = cp.asarray(data_cpu)

# Standardisation functions
def standardise_cpu(data):
    mean = np.mean(data, axis=1, keepdims=True)
    std = np.std(data, axis=1, keepdims=True)
    return (data - mean) / std

def standardise_gpu(data):
    mean = cp.mean(data, axis=1, keepdims=True)
    std = cp.std(data, axis=1, keepdims=True)
    return (data - mean) / std

data_cpu_std = standardise_cpu(data_cpu)
data_gpu_std = standardise_gpu(data_gpu)

# Correlation computation
def compute_corr_cpu(data):
    return np.dot(data, data.T) / (data.shape[1] - 1)

def compute_corr_gpu(data):
    return cp.dot(data, data.T) / (data.shape[1] - 1)

start_cpu = time.time()
corr_cpu = compute_corr_cpu(data_cpu_std)
time_cpu = time.time() - start_cpu

cp.cuda.Stream.null.synchronize()
start_gpu = time.time()
corr_gpu = compute_corr_gpu(data_gpu_std)
cp.cuda.Stream.null.synchronize()
time_gpu = time.time() - start_gpu

print(f"CPU correlation time: {time_cpu:.4f} s")
print(f"GPU correlation time: {time_gpu:.4f} s")

# Thresholding and degree centrality
def threshold_corr_cpu(corr, threshold=0.2):
    return np.where(corr >= threshold, corr, 0)

def threshold_corr_gpu(corr, threshold=0.2):
    return cp.where(corr >= threshold, corr, 0)

threshold_value = 0.2
corr_cpu_thresh = threshold_corr_cpu(corr_cpu, threshold_value)
corr_gpu_thresh = threshold_corr_gpu(corr_gpu, threshold_value)

degree_cpu = np.sum(corr_cpu_thresh, axis=1)
degree_gpu = cp.asnumpy(cp.sum(corr_gpu_thresh, axis=1))

corr_result = np.corrcoef(degree_cpu, degree_gpu)[0, 1]
print(f"Correlation between CPU and GPU degree centrality: {corr_result:.6f}")

# Plotting distributions
plt.figure(figsize=(10, 6))
sns.histplot(degree_cpu, color='blue', label='CPU', kde=True, stat='density')
sns.histplot(degree_gpu, color='red', label='GPU', kde=True, stat='density', alpha=0.7)
plt.xlabel('Degree Centrality')
plt.ylabel('Density')
plt.legend()
plt.show()

# Summary table
results = pd.DataFrame({'Implementation': ['CPU', 'GPU'], 'Time (s)': [time_cpu, time_gpu]})
print(results)